This is exercise to determinine "cheaper" technology stocks by PE
1. Get Average of PE and forward PE for technology sector in S&P500 
2. Filter target stocks by Profit margin >= 15% and below Average of PE in technology sector in S&P500
3. Or Forward PE >= 2 standard deviations of forward PE in technology sector in S&P500


In [92]:
import warnings 
warnings.filterwarnings('ignore')

#Get Average of PE in technology sector in S&P500
#Get S&P 500 stocks and sector info from Wiki
#Get PE and other metrics from Yahoo Finance API 

In [93]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Define the URL of the Wikipedia page for scraping
URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Function to make the HTTP request and obtain the content
def get_html_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Error in obtaining page content, status code: {response.status_code}")

# Function to parse HTML content with BeautifulSoup
def parse_html_to_soup(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup

# Get the HTML content of the Wikipedia page
html_content = get_html_content(URL)

# Parse the HTML content to get the BeautifulSoup object
soup = parse_html_to_soup(html_content)

# Verify that significant content was extracted
test_element = soup.find('h1', id="firstHeading")
assert test_element is not None, "The HTML content does not contain the expected element."
print("The HTML content has been successfully extracted and parsed.")

# Function to extract the data from the current S&P 500 constituents table
def extract_current_snp_data(soup):
    table = soup.find('table', {'id': 'constituents'})
    rows = table.find_all('tr')[1:]  # Exclude the table header
    current_data = []
    for row in rows:
        cols = row.find_all('td')
        symbol = cols[0].text.strip()
        security = cols[1].text.strip()
        gis_sector = cols[2].text.strip()
        gis_sub_ind = cols[3].text.strip()
        cik_code = cols[6].text.strip() #for EDGAR filing
        current_data.append({'Security': security, 'Symbol': symbol, 'GISC_Sector': gis_sector, 'GISC_SUB_INDUSTRY': gis_sub_ind, "CIK": cik_code})
    return current_data

# Extract data and create a DataFrame
current_data = extract_current_snp_data(soup)
dataframe_current = pd.DataFrame(current_data)

# Display the first records to confirm
print(dataframe_current.head())


The HTML content has been successfully extracted and parsed.
      Security Symbol             GISC_Sector               GISC_SUB_INDUSTRY  \
0           3M    MMM             Industrials        Industrial Conglomerates   
1  A. O. Smith    AOS             Industrials               Building Products   
2       Abbott    ABT             Health Care           Health Care Equipment   
3       AbbVie   ABBV             Health Care                   Biotechnology   
4    Accenture    ACN  Information Technology  IT Consulting & Other Services   

          CIK  
0  0000066740  
1  0000091142  
2  0000001800  
3  0001551152  
4  0001467373  


In [94]:
sector_list = dataframe_current['GISC_Sector'].unique()
sector_list

array(['Industrials', 'Health Care', 'Information Technology',
       'Utilities', 'Financials', 'Materials', 'Consumer Discretionary',
       'Real Estate', 'Communication Services', 'Consumer Staples',
       'Energy'], dtype=object)

In [95]:
sym_list = ' '.join(dataframe_current['Symbol'])

In [96]:
import yfinance as yf

yf_tickers = yf.Tickers(sym_list)


df_tickerInfo = pd.DataFrame()

df_list = [] #Create a list to store the ticker info, ticker info is in dict format

for t in yf_tickers.symbols:
    _ = yf_tickers.tickers[t].info
    _['sym'] = t
    df_list.append(pd.DataFrame([_]))

df_tickerInfo = pd.concat(df_list) #convert list of dict to dataframe

df_tickerInfo = pd.DataFrame(df_tickerInfo).reset_index()
df_tickerInfo


,index,address1,city,state,zip,country,phone,website,industry,industryKey,...,ebitdaMargins,operatingMargins,financialCurrency,trailingPegRatio,sym,address2,trailingPE,fax,grossProfits,industrySymbol
0,0,3M Center,Saint Paul,MN,55144-1000,United States,651 733 1110,https://www.3m.com,Conglomerates,conglomerates,...,0.24947,0.21704,USD,NaN,MMM,NaN,NaN,NaN,NaN,NaN
1,0,11270 West Park Place,Milwaukee,WI,53224-9508,United States,414 359 4000,https://www.aosmith.com,Specialty Industrial Machinery,specialty-industrial-machinery,...,0.21347,0.19657,USD,NaN,AOS,Suite 170 PO Box 245008,21.168830,NaN,NaN,NaN
2,0,100 Abbott Park Road,North Chicago,IL,60064-6400,United States,224 667 6100,https://www.abbott.com,Medical Devices,medical-devices,...,0.25710,0.15405,USD,4.1597,ABT,Abbott Park,32.133957,NaN,NaN,NaN
3,0,1 North Waukegan Road,North Chicago,IL,60064-6400,United States,847 932 7900,https://www.abbvie.com,Drug Manufacturers - General,drug-manufacturers-general,...,0.48005,0.28294,USD,0.4366,ABBV,NaN,50.044643,NaN,NaN,NaN
4,0,1 Grand Canal Square,Dublin,NaN,D02 P820,Ireland,353 1 646 2000,https://www.accenture.com,Information Technology Services,information-technology-services,...,0.17464,0.13683,USD,1.8148,ACN,Grand Canal Harbour,25.819294,353 1 646 2020,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,0,300 Water Street SE,Washington,DC,20003,United States,202 869 9150,https://www.xylem.com,Specialty Industrial Machinery,specialty-industrial-machinery,...,0.18128,0.10723,USD,2.3783,XYL,Suite 200,47.323260,NaN,NaN,NaN
499,0,1441 Gardiner Lane,Louisville,KY,40213,United States,502 874 8300,https://www.yum.com,Restaurants,restaurants,...,0.36563,0.34481,USD,2.2601,YUM,NaN,24.163120,NaN,NaN,NaN
500,0,3 Overlook Point,Lincolnshire,IL,60069,United States,847 634 6700,https://www.zebra.com,Communication Equipment,communication-equipment,...,0.15962,0.14468,USD,0.9195,ZBRA,NaN,59.300396,NaN,NaN,NaN
501,0,345 East Main Street,Warsaw,IN,46580,United States,574 373 3333,https://www.zimmerbiomet.com,Medical Devices,medical-devices,...,0.33189,0.20776,USD,1.6069,ZBH,NaN,23.140997,NaN,NaN,NaN


In [97]:
#Display data columns from yahoo finance
' '.join(df_tickerInfo.columns)

'index address1 city state zip country phone website industry industryKey industryDisp sector sectorKey sectorDisp longBusinessSummary fullTimeEmployees companyOfficers auditRisk boardRisk compensationRisk shareHolderRightsRisk overallRisk governanceEpochDate compensationAsOfEpochDate irWebsite maxAge priceHint previousClose open dayLow dayHigh regularMarketPreviousClose regularMarketOpen regularMarketDayLow regularMarketDayHigh dividendRate dividendYield exDividendDate payoutRatio fiveYearAvgDividendYield beta forwardPE volume regularMarketVolume averageVolume averageVolume10days averageDailyVolume10Day bid ask bidSize askSize marketCap fiftyTwoWeekLow fiftyTwoWeekHigh priceToSalesTrailing12Months fiftyDayAverage twoHundredDayAverage trailingAnnualDividendRate trailingAnnualDividendYield currency enterpriseValue profitMargins floatShares sharesOutstanding sharesShort sharesShortPriorMonth sharesShortPreviousMonthDate dateShortInterest sharesPercentSharesOut heldPercentInsiders heldPer

In [98]:
#Preview data before analysis
df_tickerInfo[['sym','sector','forwardPE','profitMargins','bookValue','priceToBook','mostRecentQuarter','earningsQuarterlyGrowth','trailingEps', 'forwardEps', 'pegRatio','enterpriseToRevenue', 'enterpriseToEbitda','totalCash', 'totalCashPerShare', 'ebitda', 'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 'returnOnAssets', 'returnOnEquity', 'freeCashflow', 'operatingCashflow',  'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins', 'trailingPegRatio', 'trailingPE']]

,sym,sector,forwardPE,profitMargins,bookValue,priceToBook,mostRecentQuarter,earningsQuarterlyGrowth,trailingEps,forwardEps,...,returnOnEquity,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,trailingPegRatio,trailingPE
0,MMM,Industrials,12.823090,-0.21569,8.794,11.373676,1.711843e+09,-0.049,-12.73,7.80,...,-0.69286,8.903375e+09,6.172000e+09,-0.051,-0.003,0.44550,0.24947,0.21704,NaN,NaN
1,AOS,Industrials,18.735632,0.14936,12.830,6.352299,1.711843e+09,0.163,3.85,4.35,...,0.31443,4.549750e+08,6.570000e+08,0.190,0.013,0.38689,0.21347,0.19657,NaN,21.168830
2,ABT,Healthcare,20.068094,0.13961,22.309,4.623694,1.711843e+09,-0.071,3.21,5.14,...,0.14762,4.607125e+09,7.143000e+09,-0.067,0.022,0.55265,0.25710,0.15405,4.1597,32.133957
3,ABBV,Healthcare,13.942785,0.11016,4.535,37.078280,1.711843e+09,4.728,3.36,12.06,...,0.56244,2.361150e+10,2.268600e+10,4.964,0.007,0.69174,0.48005,0.28294,0.4366,50.044643
4,ACN,Technology,21.809107,0.10889,43.132,6.608667,1.709165e+09,0.099,11.04,13.07,...,0.27381,8.528630e+09,9.298353e+09,0.101,-0.001,0.32578,0.17464,0.13683,1.8148,25.819294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,Industrials,28.632563,0.08341,42.041,3.241859,1.711843e+09,0.545,2.88,4.76,...,0.09614,5.701250e+08,9.450000e+08,0.167,0.404,0.36772,0.18128,0.10723,2.3783,47.323260
499,YUM,Consumer Cyclical,21.394035,0.22919,-27.601,NaN,1.711843e+09,0.047,5.64,6.37,...,NaN,1.170750e+09,1.617000e+09,0.048,-0.029,0.49595,0.36563,0.34481,2.2601,24.163120
500,ZBRA,Technology,20.426140,0.05994,61.654,4.866837,1.711757e+09,-0.233,5.06,14.69,...,0.08614,1.483750e+08,1.970000e+08,-0.231,-0.164,0.46371,0.15962,0.14468,0.9195,59.300396
501,ZBH,Healthcare,12.276180,0.12934,61.245,1.741856,1.711843e+09,-0.259,4.61,8.69,...,0.07795,1.070800e+09,1.501900e+09,-0.243,0.032,0.71884,0.33189,0.20776,1.6069,23.140997


In [99]:
#joining symbol infos
df_tickerInfo = pd.DataFrame(df_tickerInfo).reset_index()
df_tickerInfo = dataframe_current.merge(df_tickerInfo, left_on='Symbol', right_on='sym')
df_tickerInfo

,Security,Symbol,GISC_Sector,GISC_SUB_INDUSTRY,CIK,level_0,index,address1,city,state,...,ebitdaMargins,operatingMargins,financialCurrency,trailingPegRatio,sym,address2,trailingPE,fax,grossProfits,industrySymbol
0,3M,MMM,Industrials,Industrial Conglomerates,0000066740,0,0,3M Center,Saint Paul,MN,...,0.24947,0.21704,USD,NaN,MMM,NaN,NaN,NaN,NaN,NaN
1,A. O. Smith,AOS,Industrials,Building Products,0000091142,1,0,11270 West Park Place,Milwaukee,WI,...,0.21347,0.19657,USD,NaN,AOS,Suite 170 PO Box 245008,21.168830,NaN,NaN,NaN
2,Abbott,ABT,Health Care,Health Care Equipment,0000001800,2,0,100 Abbott Park Road,North Chicago,IL,...,0.25710,0.15405,USD,4.1597,ABT,Abbott Park,32.133957,NaN,NaN,NaN
3,AbbVie,ABBV,Health Care,Biotechnology,0001551152,3,0,1 North Waukegan Road,North Chicago,IL,...,0.48005,0.28294,USD,0.4366,ABBV,NaN,50.044643,NaN,NaN,NaN
4,Accenture,ACN,Information Technology,IT Consulting & Other Services,0001467373,4,0,1 Grand Canal Square,Dublin,NaN,...,0.17464,0.13683,USD,1.8148,ACN,Grand Canal Harbour,25.819294,353 1 646 2020,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,Xylem Inc.,XYL,Industrials,Industrial Machinery & Supplies & Components,0001524472,498,0,300 Water Street SE,Washington,DC,...,0.18128,0.10723,USD,2.3783,XYL,Suite 200,47.323260,NaN,NaN,NaN
499,Yum! Brands,YUM,Consumer Discretionary,Restaurants,0001041061,499,0,1441 Gardiner Lane,Louisville,KY,...,0.36563,0.34481,USD,2.2601,YUM,NaN,24.163120,NaN,NaN,NaN
500,Zebra Technologies,ZBRA,Information Technology,Electronic Equipment & Instruments,0000877212,500,0,3 Overlook Point,Lincolnshire,IL,...,0.15962,0.14468,USD,0.9195,ZBRA,NaN,59.300396,NaN,NaN,NaN
501,Zimmer Biomet,ZBH,Health Care,Health Care Equipment,0001136869,501,0,345 East Main Street,Warsaw,IN,...,0.33189,0.20776,USD,1.6069,ZBH,NaN,23.140997,NaN,NaN,NaN


In [100]:
#Calculate mean, min, max for sector metrics (['forwardEps','trailingPE','forwardPE','profitMargins','revenuePerShare','returnOnEquity', 'debtToEquity','pegRatio')

x_sector_df_list_Metrics_Desc = {}

for _ in sector_list:
    x_sector_df_list_Metrics_Desc[_] = df_tickerInfo[df_tickerInfo['GISC_Sector'].isin([_])][['forwardEps','trailingPE','forwardPE','profitMargins','revenuePerShare','returnOnEquity', 'debtToEquity','pegRatio']].describe()



In [101]:
#show Metrics for Technology sector
x_sector_df_list_Metrics_Desc['Information Technology']

,forwardEps,trailingPE,forwardPE,profitMargins,revenuePerShare,returnOnEquity,debtToEquity,pegRatio
count,65.000000,61.000000,65.000000,65.000000,65.000000,60.000000,58.000000,65.000000
mean,11.210308,44.550481,24.250874,0.174134,45.549338,0.387604,111.886569,0.817385
std,9.631314,32.460077,10.444792,0.126044,45.140809,0.563412,212.788132,8.545902
min,1.950000,10.305026,9.622642,-0.205710,3.234000,-0.133540,0.405000,-58.120000
25%,5.590000,25.463541,16.121168,0.094970,18.488000,0.098508,22.545750,1.700000
50%,8.450000,39.142857,21.809107,0.176620,31.795000,0.264130,43.764500,2.160000
75%,13.750000,55.119766,30.013456,0.240820,53.425000,0.370620,97.151000,3.030000
max,58.870000,230.775360,51.462032,0.550560,247.585000,3.554710,1280.447000,7.370000


Get and preview stock metrics for target stock list

In [102]:
target_sym_list = ['NVDA','GOOGL','META','ADBE','AMD','QCOM','MU','ISRG','INTC','CDNS','EQIX','ADSK','SNPS','DLR','TTD','SNOW','MRVL','MCHP','SMCI','HUBS','AME','ANSS','PINS','WDC','MSTR','STX','EA','FTV','TTWO','NTAP','HPE','GRMN','PTC','PSTG','NICE','BSY','LSCC','ESTC','CGNX','ROKU','NXT','DLB','ALTR','U','ALU','NOVT','SLAB','NEM','DIOD','RDNT','TDC','FIVN','IPGP','ZM','BB']
target_yf_tickers = yf.Tickers(target_sym_list)


target_df_tickerInfo = pd.DataFrame()

target_df_list = [] #Create a list to store the ticker info, ticker info is in dict format

for t in target_yf_tickers.symbols:
    _ = target_yf_tickers.tickers[t].info
    _['sym'] = t
    target_df_list.append(pd.DataFrame([_]))

target_df_tickerInfo = pd.concat(target_df_list) #convert list of dict to dataframe

target_df_tickerInfo = pd.DataFrame(target_df_tickerInfo).reset_index()
target_df_tickerInfo.head(10)

,index,address1,city,state,zip,country,phone,website,industry,industryKey,...,earningsGrowth,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,financialCurrency,trailingPegRatio,sym,fax,address2
0,0,2788 San Tomas Expressway,Santa Clara,CA,95051,United States,408 486 2000,https://www.nvidia.com,Semiconductors,semiconductors,...,6.500,2.621,0.75286,0.61768,0.64925,USD,1.4856,NVDA,NaN,NaN
1,0,1600 Amphitheatre Parkway,Mountain View,CA,94043,United States,650 253 0000,https://abc.xyz,Internet Content & Information,internet-content-information,...,0.609,0.154,0.57466,0.34488,0.32516,USD,1.5772,GOOGL,NaN,NaN
2,0,1 Meta Way,Menlo Park,CA,94025,United States,650 543 4800,https://investor.fb.com,Internet Content & Information,internet-content-information,...,1.141,0.273,0.81496,0.47962,0.38579,USD,1.1650,META,NaN,NaN
3,0,345 Park Avenue,San Jose,CA,95110-2704,United States,408 536 6000,https://www.adobe.com,Software - Infrastructure,software-infrastructure,...,-0.498,0.113,0.88082,0.38067,0.36800,USD,1.7024,ADBE,408 537 6000,NaN
4,0,2485 Augustine Drive,Santa Clara,CA,95054,United States,408 749 4000,https://www.amd.com,Semiconductors,semiconductors,...,NaN,0.022,0.50561,0.16829,0.00658,USD,1.4670,AMD,NaN,NaN
5,0,5775 Morehouse Drive,San Diego,CA,92121-1714,United States,858 587 1121,https://www.qualcomm.com,Semiconductors,semiconductors,...,0.357,0.012,0.55805,0.29766,0.24923,USD,1.2853,QCOM,NaN,NaN
6,0,8000 South Federal Way,Boise,ID,83716-9632,United States,208 368 4000,https://www.micron.com,Semiconductors,semiconductors,...,NaN,0.577,-0.00322,0.20003,0.03280,USD,NaN,MU,208 368 4617,NaN
7,0,1020 Kifer Road,Sunnyvale,CA,94086-5304,United States,408 523 2100,https://www.intuitive.com,Medical Instruments & Supplies,medical-instruments-supplies,...,0.510,0.115,0.66433,0.30982,0.24828,USD,4.4528,ISRG,408 523 1390,NaN
8,0,2200 Mission College Boulevard,Santa Clara,CA,95054-1549,United States,408 765 8080,https://www.intel.com,Semiconductors,semiconductors,...,NaN,0.086,0.41494,0.19011,-0.05666,USD,0.5455,INTC,NaN,NaN
9,0,Building 5,San Jose,CA,95134,United States,408 943 1234,https://www.cadence.com,Software - Application,software-application,...,0.022,-0.012,0.89313,0.33322,0.25690,USD,2.8661,CDNS,408 428 5001,2655 Seely Avenue


#Filter Condition
#Profit margin >= 15% & PE below avg of PE of tech stocks in S&P 500
#Forward PE above avg of forward PE of tech stocks in S&P 500

In [103]:
print(f"Avg of PE of tech stocks in S&P500 { x_sector_df_list_Metrics_Desc['Information Technology']['trailingPE']['mean']} ")
std2 = x_sector_df_list_Metrics_Desc['Information Technology']['forwardPE']['mean'] + (2 * x_sector_df_list_Metrics_Desc['Information Technology']['forwardPE']['std'])
print(f"2 standard deviation of Forward PE of tech stocks in S&P500 { std2 } ")

Avg of PE of tech stocks in S&P500 44.550481311475416 
2 standard deviation of Forward PE of tech stocks in S&P500 45.14045894173389 


In [106]:
#filter the result

filtered_result = target_df_tickerInfo[((target_df_tickerInfo['profitMargins'] >= 0.15) & (target_df_tickerInfo['trailingPE'] <= x_sector_df_list_Metrics_Desc['Information Technology']['trailingPE']['mean']) ) |  (target_df_tickerInfo['forwardPE'] >= std2)][['sym','forwardEps','trailingPE','forwardPE','profitMargins','revenuePerShare','returnOnEquity', 'debtToEquity','pegRatio']]
print(f' # of stocks by filters: {len(filtered_result)}' )
filtered_result.sort_values('sym')

 # of stocks by filters: 22


,sym,forwardEps,trailingPE,forwardPE,profitMargins,revenuePerShare,returnOnEquity,debtToEquity,pegRatio
42,ALTR,1.39,878.272770,69.503600,0.01546,7.631,0.01399,44.392,1.56
20,AME,7.47,29.067030,22.140750,0.19573,29.190,0.15766,35.314,2.92
54,BB,0.03,NaN,79.000000,-0.15240,1.459,-0.15922,32.474,0.87
41,DLB,3.91,39.932990,19.813300,0.15003,13.272,0.08015,2.130,1.31
13,DLR,1.34,41.237570,111.402985,0.21477,17.800,0.05847,88.764,-8.37
26,EA,8.36,28.910446,16.218900,0.16834,28.007,0.17196,29.229,1.51
10,EQIX,11.97,76.708120,63.891396,0.11939,83.833,0.07728,140.190,6.47
37,ESTC,1.82,191.546620,62.094776,0.04870,12.718,0.10856,80.427,4.96
1,GOOGL,8.60,27.161291,20.560465,0.25902,25.374,0.29764,9.690,1.17
31,GRMN,6.18,22.477346,25.787119,0.24957,28.525,0.19973,3.610,3.56
